Ссылка на оригинальную статью [здесь](https://medium.com/the-andela-way/on-genetic-algorithms-and-their-application-in-solving-regression-problems-4e37ac1115d5)

1. individual (индивид) - возможное решение
2. population (популяция) - набор возможных решений
3. fitness (фитнес) - способ измерить, насколько решение хорошее
4. generation (поколение) - насколько сильно мы продвинулись в сторону желаемой функции fitness
5. variation operation (вариативная операция) - вносимое изменение, требуемое для изменения результата

1. Хромосома: набор связанных генов, несущих генетическую информацию. Это может быть участок математической функции с несколькими параметрами
2. Кроссовер: получение потомства путем объединения родителей. Он может сформировать новую математическую функцию, комбинируя наборы параметров из других функций. Одна из основных вариационных операций, упомянутых выше
3. Ген: носитель генетической информации, касающейся определенного признака, другими словами, единицы наследственности. Это может быть параметр математической функции, влияющий на конкретную переменную
4. Геном: полная комбинация генетической информации для индивида. Это может быть математическая функция целиком
5. Фитнес: мера адаптации индивида. Это может быть тем, насколько хорошо работает решение проблемы оптимизации
6. Локус: определенное место на хромосоме, например, местоположение гена. Это может быть расположение параметра в математической функции
7. Мутация: изменение последовательности гена. Это может быть изменение параметра математической функции, приводящее к изменению получаемых ею результатов. Также вариативная операция
8. Популяция: совокупность всех индивидов, рассматриваемых в исследовании. Это может быть набор из множества решений проблемы оптимизации, из которых нужно выбрать лучшее

1. Критерий остановки: способ остановить алгоритм, если решение было найдено, достигнут предел разрешенных поколений или любые другие критерии
2. Отбор: метод выбора индивидов, которые будут использоваться при создании следующего поколения
3. Вариация: это применение изменений к избранным индивидам, чтобы изменить их поведение. Это делается либо с помощью мутациии, либо кроссовером

In [1]:
from random import random, sample, choice
from math import floor
from tqdm import tqdm
from numpy import array, dot, mean
from numpy.linalg import pinv
import numpy as np

In [2]:
# пишем генератор данных
def generate_data():
    coeff = [0.4, -0.3, 0.2, -0.1]
    x = [[random() for j in range(len(coeff))] for i in range(1000)]
    y = [dot(i, coeff) for i in x]
    return array(x), array(y)

1. SST: общая ошибка модели, это сумма всех отклонений в квадрате
2. SSR: мера объясненного изменения SST.
3. COD: означает «коэффициент детерминации», который в основном является мерой того, насколько хороша модель. Он представляет собой часть дисперсии зависимой переменной (выход), которая объясняется дисперсией в независимых переменных (входных данных). Диапазон значений от 0 до 100, любое другое значение означает, что модель плохая. В допустимом диапазоне 0 - относительно плохая модель, а 100 - наилучшая из возможных.
4. error: средняя ошибка, представляет собой среднее значение всех отклонений от ожидаемых значений

In [3]:
def multiple_linear_regression(inputs, outputs):
    X, Y = array(inputs), array(outputs)
    X_t, Y_t = X.transpose(), Y.transpose()
    coeff = dot((pinv((dot(X_t, X)))), (dot(X_t, Y)))
    Y_p = dot(X, coeff)
    Y_mean = mean(Y)
    SST = array([(i - Y_mean) ** 2 for i in Y]).sum()
    SSR = array([(i - j) ** 2 for i, j in zip(Y, Y_p)]).sum()
    COD = (1 - (SSR / SST)) * 100.0
    av_error = (SSR / len(Y))
    return {'COD': COD, 'coeff': coeff, 'error': av_error}

In [4]:
# критерий остановки
def check_termination_condition(best_individual):
    if ((best_individual['COD'] >= 98.0)
            or (generation_count == max_generations)):
        return True
    else:
        return False

In [5]:
# создаем случайного индивида
def create_individual(individual_size):
    return [random() for i in range(individual_size)]

In [6]:
# создаем популяцию, тоже случайным образом
def create_population(individual_size, population_size):
    return [create_individual(individual_size) for i in range(population_size)]

In [7]:
# пишем функцию фитнес
def get_fitness(individual, inputs):
    predicted_outputs = np.array(inputs).dot(np.array(individual)) #dot(array(inputs, array(individual)))
    output_mean = mean(outputs)
    SST = array(
        [(i - output_mean) ** 2 for i in outputs]
        ).sum()
    SSR = array(
        [(i - j) ** 2 for i, j in zip(outputs, predicted_outputs)]
        ).sum()
    COD = (1 - (SSR / SST)) * 100.0
    average_error = (SSR / len(outputs))
    return {'COD': COD, 'error': average_error, 'coeff': individual}

In [8]:
# используем эту функцию, чтобы отбирать лучших индивидов
def evaluate_population(population):
    fitness_list = [get_fitness(individual, inputs)
                    for individual in tqdm(population)]
    error_list = sorted(fitness_list, key=lambda i: i['error'])
    best_individuals = error_list[: selection_size]
    best_individuals_stash.append(best_individuals[0]['coeff'])
    print('Error: ', best_individuals[0]['error'],
          'COD: ', best_individuals[0]['COD'])
    return best_individuals

In [9]:
# Для эволюции популяции необходимо применять операции вариации. Мы будем использовать кроссовер и мутацию
# При кроссовере два родителя передают гены ребенку, каждый дает половину
def crossover(parent_1, parent_2):
    child = {}
    loci = [i for i in range(0, individual_size)]
    loci_1 = sample(loci, floor(0.5*(individual_size)))
    loci_2 = [i for i in loci if i not in loci_1]
    chromosome_1 = [[i, parent_1['coeff'][i]] for i in loci_1]
    chromosome_2 = [[i, parent_2['coeff'][i]] for i in loci_2]
    child.update({key: value for (key, value) in chromosome_1})
    child.update({key: value for (key, value) in chromosome_2})
    return [child[i] for i in loci]

In [10]:
# При мутации выбранное количество генов у отобранного количества индивидов изменяется случайным образом
def mutate(individual):
    loci = [i for i in range(0, individual_size)]
    no_of_genes_mutated = floor(probability_of_gene_mutating*individual_size)
    loci_to_mutate = sample(loci, no_of_genes_mutated)
    for locus in loci_to_mutate:
        gene_transform = choice([-1, 1])
        change = gene_transform*random()
        individual[locus] = individual[locus] + change
    return individual

In [11]:
# создаем новое поколение индивидов из лучших индивидов
def get_new_generation(selected_individuals):
    parent_pairs = [sample(selected_individuals, 2)
                    for i in range(population_size)]
    offspring = [crossover(pair[0], pair[1]) for pair in parent_pairs]
    offspring_indices = [i for i in range(population_size)]
    offspring_to_mutate = sample(
        offspring_indices,
        floor(probability_of_individual_mutating*population_size)
    )
    mutated_offspring = [[i, mutate(offspring[i])]
                         for i in offspring_to_mutate]
    for child in mutated_offspring:
        offspring[child[0]] = child[1]
    return offspring

In [12]:
inputs, outputs = generate_data()
individual_size = len(inputs[0])
population_size = 1000
selection_size = floor(0.1*population_size)
max_generations = 50
probability_of_individual_mutating = 0.1
probability_of_gene_mutating = 0.25
best_possible = multiple_linear_regression(inputs, outputs)
best_individuals_stash = [create_individual(individual_size)]
initial_population = create_population(individual_size, 1000)
current_population = initial_population
termination = False
generation_count = 0
while termination is False:
    current_best_individual = get_fitness(best_individuals_stash[-1], inputs)
    print('Generation: ', generation_count)
    best_individuals = evaluate_population(current_population)
    current_population = get_new_generation(best_individuals)
    termination = check_termination_condition(current_best_individual)
    generation_count += 1
else:
    print(get_fitness(best_individuals_stash[-1], inputs))

  7%|▋         | 70/1000 [00:00<00:01, 696.42it/s]

Generation:  0


  7%|▋         | 72/1000 [00:00<00:01, 715.71it/s]

Error:  0.027209297389456752 COD:  -8.944286975742388
Generation:  1


  7%|▋         | 67/1000 [00:00<00:01, 661.85it/s]

...

Error:  0.0005130736770728578 COD:  97.94568580310406
Generation:  10


  6%|▋         | 64/1000 [00:00<00:01, 634.52it/s]

Error:  0.00039920180253312 COD:  98.40162150775514
Generation:  11


100%|██████████| 1000/1000 [00:01<00:00, 665.18it/s]


Error:  0.0003959647998600512 COD:  98.41458225948307
{'COD': 98.41458225948307, 'error': 0.0003959647998600512, 'coeff': [0.4089559704890152, -0.3082798032802887, 0.15637119737277827, -0.049306826564520345]}


In [13]:
!wget "https://docs.google.com/uc?export=download&id=1bWp01DceEwx5nXZ1cnuaPod9AV1nEjaf" -O data.csv

--2021-01-05 21:07:20--  https://docs.google.com/uc?export=download&id=1bWp01DceEwx5nXZ1cnuaPod9AV1nEjaf
Resolving docs.google.com (docs.google.com)... 64.233.170.100, 64.233.170.102, 64.233.170.113, ...
Connecting to docs.google.com (docs.google.com)|64.233.170.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uk4f5b6bntrhr14pqene5mjedtlf6dou/1609880775000/01540819630670776734/*/1bWp01DceEwx5nXZ1cnuaPod9AV1nEjaf?e=download [following]
--2021-01-05 21:07:21--  https://doc-08-0g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uk4f5b6bntrhr14pqene5mjedtlf6dou/1609880775000/01540819630670776734/*/1bWp01DceEwx5nXZ1cnuaPod9AV1nEjaf?e=download
Resolving doc-08-0g-docs.googleusercontent.com (doc-08-0g-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-08-0g-docs.googleusercontent.com (doc-08-0g-

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import seaborn as sns
import time
data = pd.read_csv('data.csv')
x = np.array(data.drop(columns=['Serial No.', 'Chance of Admit ']))
y = np.array(data['Chance of Admit '])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
x_train, x_test = MinMaxScaler().fit(x).transform(x_train), MinMaxScaler().fit(x).transform(x_test)

In [32]:
# отбираем признаки
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(x_train, y_train)
model.feature_importances_

array([0.05206423, 0.01718461, 0.01463759, 0.03381826, 0.02657914,
       0.84326305, 0.01245312])

In [33]:
data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [34]:
inputs, outputs = x_train[:, [0, 3, 4, 5]], y_train
individual_size = len(inputs[0])
def gen():
  for prob_mut in [0.1, 0.2, 0.3]:
    for prob_gene in [0.15, 0.25, 0.35]:
      population_size = 10000
      selection_size = floor(0.1*population_size)
      max_generations = 50
      probability_of_individual_mutating = prob_mut
      probability_of_gene_mutating = prob_gene
      best_possible = multiple_linear_regression(inputs, outputs)
      best_individuals_stash = [create_individual(individual_size)]
      initial_population = create_population(individual_size, 1000)
      current_population = initial_population
      termination = False
      generation_count = 0
      while termination is False and generation_count != max_generations:
          current_best_individual = get_fitness(best_individuals_stash[-1], inputs)
          print('Generation: ', generation_count)
          best_individuals = evaluate_population(current_population)
          current_population = get_new_generation(best_individuals)
          termination = check_termination_condition(current_best_individual)
          generation_count += 1
          if best_individuals[0]['COD'] >= 98:
            print('BEST INDIVIDUALS COD:', best_individuals[0]['COD'])
            print(f'BEST INDIVIDUALS PARAMS: pop size: {population_size}, selection size: {selection_size}, prob mut: {prob_mut}, prob gene: {prob_gene}')
            return best_individuals_stash[-1]
  return best_individuals_stash[-1]

In [35]:
gen()




  0%|          | 0/1000 [00:00<?, ?it/s]


 15%|█▌        | 152/1000 [00:00<00:00, 1512.82it/s]




Generation:  0


 34%|███▍      | 344/1000 [00:00<00:00, 1614.12it/s]


 53%|█████▎    | 531/1000 [00:00<00:00, 1681.73it/s]


 72%|███████▏  | 722/1000 [00:00<00:00, 1742.18it/s]


100%|██████████| 1000/1000 [00:00<00:00, 1827.75it/s]



  0%|          | 0/1000 [00:00<?, ?it/s]


 18%|█▊        | 181/1000 [00:00<00:00, 1806.05it/s]

Error:  0.013845126371752114 COD:  30.60693360115645
Generation:  1





 37%|███▋      | 371/1000 [00:00<00:00, 1830.90it/s]


 54%|█████▍    | 544/1000 [00:00<00:00, 1797.13it/s]


 73%|███████▎  | 727/1000 [00:00<00:00, 1805.64it/s]


100%|██████████| 1000/1000 [00:00<00:00, 1812.54it/s]



  0%|          | 0/1000 [00:00<?, ?it/s]


 20%|██        | 201/1000 [00:00<00:00, 2007.60it/s]

...

Error:  0.013367342249828981 COD:  33.00163224143541
Generation:  3


 34%|███▍      | 339/1000 [00:00<00:00, 1668.37it/s]


 53%|█████▎    | 529/1000 [00:00<00:00, 1730.24it/s]


 72%|███████▏  | 724/1000 [00:00<00:00, 1790.72it/s]


100%|██████████| 1000/1000 [00:00<00:00, 1788.75it/s]



  0%|          | 0/1000 [00:00<?, ?it/s]


 18%|█▊        | 185/1000 [00:00<00:00, 1844.47it/s]

Error:  0.013261298463624366 COD:  33.533133601531
Generation:  4





 36%|███▌      | 355/1000 [00:00<00:00, 1795.91it/s]


 51%|█████     | 511/1000 [00:00<00:00, 1717.43it/s]


 70%|███████   | 704/1000 [00:00<00:00, 1775.36it/s]

KeyboardInterrupt: ignored

In [36]:
pred_real = pd.DataFrame({"pred":best_individuals_stash[-1][0] * x[:, 0] + best_individuals_stash[-1][1] * x[:, 1] + best_individuals_stash[-1][2] * x[:, 2] + + best_individuals_stash[-1][3] * x[:, 3], "real": y})

In [37]:
from sklearn.metrics import mean_squared_error
mean_squared_error(pred_real['real'], pred_real['pred'])

516.9741921785861

**Домашнее задание**. Попробовать улучшить этот результат.